In [42]:
from google.colab import drive
drive.mount('/content/drive/')
import os
path="/content/drive/My Drive/KnowledgeGraph/KG_Constraction/Model3"

os.chdir(path)
os.listdir(path)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


['entity-aware-relation-classification',
 'Model3_Run3_RE_BiLSTM.ipynb',
 'Model3_Run1.ipynb',
 'FewRel',
 'Model3_Run4_FewShot.ipynb',
 'Model3_Run2_Visualization.ipynb',
 'Model3_Run7_Visualization.ipynb',
 'Model3_Run5_DenpendencePath.ipynb',
 'Model3_Run9.ipynb',
 'exp_data.csv',
 'sentence_1.csv',
 'sentence_2.csv',
 'sentence_3.csv',
 'sentence_4.csv',
 'sentence_5.csv',
 'sentence_6.csv',
 'sentence_7.csv',
 'sentence_8.csv',
 'Model3_Run6_CSDN.ipynb',
 'Model3_Run8_OIE.ipynb',
 'Model3_Run6_Advanced_CSDN.ipynb',
 'sentence_9.csv']

In [ ]:
!python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 778.8 MB 17 kB/s 
  Attempting uninstall: en-core-web-lg
    Found existing installation: en-core-web-lg 3.4.0
    Uninstalling en-core-web-lg-3.4.0:
      Successfully uninstalled en-core-web-lg-3.4.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
! pip install allennlp==2.1.0 allennlp-models==2.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 4.3 MB/s 
     |████████████████████████████████| 407 kB 36.4 MB/s 
     |████████████████████████████████| 1.2 MB 50.4 MB/s 
     |████████████████████████████████| 5.8 MB 27.1 MB/s 
     |████████████████████████████████| 1.9 MB 47.1 MB/s 
     |████████████████████████████████| 132 kB 71.4 MB/s 
     |████████████████████████████████| 592 kB 65.2 MB/s 
     |████████████████████████████████| 12.8 MB 38.6 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 125 kB 63.0 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 56 kB 5.2 MB/s 
     |████████████████████████████████| 79 kB 6.2 MB/s 
     |████████████████████████████████| 9.0 MB 38.5 MB/s 
     |████████████████████████████████| 139 kB 64.7 MB/s 
     |███████████████████████████████

In [ ]:
#import neuralcoref
import spacy
from spacy import displacy
import os
import pandas as pd
import re

nlp = spacy.load("en_core_web_lg")
#neuralcoref.add_to_pipe(nlp)

In [ ]:
import pandas as pd

# 现代码

In [ ]:
# 读取 tragger 处理好的数据
# 返回 分块后的句子
# 给我一个单词+句子，返回该单词的token
def get_token(word,sentence):
  doc = nlp(sentence)
  for token in doc:
    if token.text == word:
      return token

def get_nodes_links(df_file):
  df_out = pd.read_csv(df_file)
  nodes_links = []      #每一个元素为 [noun chunk] or [link,1]

  key = -1
  com_chunk = ''
  for index in range(len(df_out['Document#'])):
    row_info = df_out.iloc[index]
    chunk = row_info[3]   # chunk name
    entity_type = str(row_info[4])
    pos = row_info[7]

    # 放入list里
    if entity_type == 'ENTITY':     #只要entity type 是"ENTITY"就认为是nodes
      nodes_links.append([chunk,index,0])
    if 'VERB' in pos or 'VERB' in entity_type:           # POS = "VERB"则节点为动词
      nodes_links.append([chunk,index,1])
    if pos=='ADP' :
      nodes_links.append([chunk,index,2])      # pos为ADP则节点为介词
  return nodes_links



In [ ]:
get_nodes_links(pd_data)

[['We', 0, 0],
 ['view', 1, 1],
 ['this problem', 2, 0],
 ['as', 3, 2],
 ['a link prediction task', 4, 0],
 ['in', 5, 2],
 ['a knowledge graph', 6, 0],
 ['show that', 9, 1],
 ['a new model of the top discriminative meta paths', 10, 0],
 ['is', 11, 1],
 ['to understand', 13, 1],
 ['the meaning of some statement', 14, 0],
 ['determine', 17, 1],
 ['its veracity', 18, 0]]

# 后处理

In [ ]:
# 三元组后处理：将节点和边进行瘦身
def clean_chunks(chunk,lemma):
    # 去掉a,an,the，','
    list1 = ['a','an','the','this','these','our','their']
    if lemma in list1:
      chunk = re.sub(lemma,'',chunk)
      chunk = re.sub(',',' ',chunk)
    chunk = chunk.lstrip()

    #去掉括号里的内容
    start_kuohao = 0
    end_kuohao = 0 
    if "(" in chunk:
      for index in range(0,len(chunk)):
        if chunk[index] == '(':
          start_kuohao = index
        if chunk[index] == ')':
          end_kuohao = index
    if start_kuohao !=0 or end_kuohao != 0:
      chunk = re.sub(chunk[start_kuohao:end_kuohao+1],'',chunk)
      chunk = chunk.replace('(','').replace(')','')
    chunk = chunk.lstrip()

    #去掉of及之前的字符串
    of_list = []
    for index in range(1,len(chunk)-3):
      if chunk[index-1]==' ' and chunk[index] == 'o' and chunk[index+1] == 'f' and chunk[index+2]==' ':
        of_list.append(index)
    if len(of_list)>0:
      #print(chunk)
      of_index = of_list[-1]
      chunk = chunk[of_index+3:]
    return chunk

# Version 2

In [ ]:
# verb ---> subj./obj.
def get_verb_sub(verb,result_dict,doc):
  key = verb
  value = result_dict[key]
  verb_token = value[3]

  
  # 判断动词是否为动名词
  if value[3].dep_ == 'csubj':  # 判断动词是否为动名词作主语的情况，如果是的话，只需找宾语即可
      for child in value[3].children:
        if child.dep_ == 'dobj':  # 当有直接的宾语时
          result_dict[key] = ['',child.text,value[2],value[3],1]
  #print('动名词：')
  #print(result_dict)
  #print()

  # 判断能否世界找出nsub,dobj
  if result_dict[key][4] != 1:  
      subject_ = ''
      object_ = ''
      for child in verb_token.children:
        #print(child.text,child.dep_)
        if child.dep_ == 'nsubj':
          subject_ = child.text                            
        if child.dep_ == 'dobj':   # 抽出动词的宾语，注意主语，宾语也一定会是动词的下位词
          object_ = child.text
      if len(subject_) > 0 and len(object_) > 0:
        result_dict[key] = [subject_,object_,value[2],value[3],1] 
  #print('直接找：')
  #print(result_dict)
  #print()

  # 判断动词的主语是否为动名词
  if result_dict[key][4] != 1: # 目前还没有主语+宾语的动词
      subject_ = ''
      object_ = ''
      for child in verb_token.children:
        #print(child.text,child.dep_)
        if child.dep_ == 'csubj':
          subject_ = child.text+ '+' + result_dict[child.text][1]
        if child.dep_ == 'dobj':
          object_ = child.text
      #print('subject:',subject_)
      if len(subject_) != 0 and len(object_) != 0:
        result_dict[key] = [subject_,object_,value[2],value[3],1]
      elif len(subject_) != 0:
        result_dict[key] = [subject_,object_,value[2],value[3],-1]
  #print('动名词是主语：')
  #print(result_dict)
  #print()

  # 判断动词是否和其他的动词并列
  if result_dict[key][4] != 1 and value[3].dep_ == 'conj':
      #print(key,value[3].dep_)
      subject_ = ''
      object_ = ''
      subject_ = result_dict[verb_token.head.text][0]
      for child in verb_token.children:
        #print(verb_token.text,child.text,child.dep_)
        if child.dep_ == 'dobj':
          object_ = child.text
      #print(key,subject_,object_)
      if len(subject_)  > 0 and len(object_) > 0:
        result_dict[key] = [subject_,object_,value[2],value[3],1]
      elif len(subject_) > 0:
        result_dict[key] = [subject_,object_,value[2],value[3],-1]
      #print(result)
  #print('并列主语：')
  #print(result_dict)
  #print()

  # 动词+介词：直接连着的情况, 动词的sub好找，宾语靠介词找
  if result_dict[key][4] != 1:
    prep_token = doc[result_dict[key][2] + 1 ]
    sub_mid = doc[result_dict[key][2] - 1]
    #情况一，n + verb_ed + prep + n
    if verb_token.dep_ == 'acl' and prep_token.dep_ == 'prep' and sub_mid.pos_=='NOUN':
      subject_ = sub_mid.text
      object_ = ''
      for child in prep_token.children:
        if child.dep_ == 'pobj':
          object_ = child.text
      if object_ != '':
        result_dict[key] = [subject_,prep_token.text+'+'+object_,value[2],value[3],1]
    # 情况二： subj.(已找到) + verb + prep + n.
    current_triple = result_dict[key]
    if current_triple[0] != '' and current_triple[1] == '' and prep_token.dep_=='prep':
      object_ = ''
      for child in prep_token.children:
        if child.dep_ == 'pobj':
          object_ = child.text
      if object_ != '':
        result_dict[key] = [current_triple[0],prep_token.text+'+'+object_,value[2],value[3],1]

  #print('verb_ed + prep:')
  #print(result_dict)
  #print()
  # 判断动词是否和其他的动词并列(是那种状语型并列)
  if result_dict[key][4] != 1 and value[3].dep_ == 'advcl':
      #print(key,value[3].dep_)
      subject_ = ''
      object_ = ''
      subject_ = result_dict[verb_token.head.text][0]
      for child in verb_token.children:
        #print(verb_token.text,child.text,child.dep_)
        if child.dep_ == 'dobj':
          object_ = child.text
      #print(key,subject_,object_)
      if len(subject_)  > 0 and len(object_) > 0:
        result_dict[key] = [subject_,object_,value[2],value[3],1]
      elif len(subject_) > 0:
        result_dict[key] = [subject_,object_,value[2],value[3],-1]
      #print(result)

  # 最后如果还是没能拿到主语，只能使用version 1, 暴力往前推
  if result_dict[key][4] != 1 :  
      subject_ = ''
      object_ = ''
      gate = -1
      #print(key,result_dict[key][3].dep_)
      for i in range(result_dict[key][2])[::-1]:
        #print(doc[i].text,doc[i].pos_)
        if doc[i].pos_ == 'NOUN' and gate == -1:
          subject_ = doc[i].text
          gate = 0
          #print(subject_)
      for child in verb_token.children:
        if child.dep_ == 'dobj':
          object_ = child.text
      result_dict[key] = [subject_,object_,value[2],value[3],1]
  #print('version 1:')
  #print(result_dict)
  #print()

  return result_dict

def get_verbs_sub(sentence):
  result = {}
  doc = nlp(sentence)

  index = 0
  # 创建所有动词的token列表和text列表
  for verb_token in doc:
    if verb_token.pos_ == "VERB":
      result[verb_token.text] = ['','',index,verb_token,-1]
      #result.append({verb_token.text:['','',index,verb_token,-1]}) #[subject,object,index,verb_token,done]
    index += 1  # 记录动词是在句子的第几个单词位置

  for key,value in result.items():
    #print("Now, for : ",key)
    result = get_verb_sub(key,result,doc)
  
  #删掉没有宾语的动词
  result_clean = {}
  for key,value in result.items():
    if len(value[0]) != 0 and len(value[1]) != 0:
      result_clean[key] = value
  #print(result_clean)

  return result_clean
 
  

In [ ]:
# prep -- > subj. + obj.
def get_prep_sub(sentence):
  result = {}
  doc = nlp(sentence)

  index = 0 
  # 创建所有的介词列表
  for prep_token in doc:
    if prep_token.text.lower() in ['in','on','at','by','with','from','as']:
      result[prep_token.text] = ['','',index,prep_token,-1]
    index += 1
  #print('the prep list is:',result)

  # noun + prep + noun 这种最理想的情况
  for key,value in result.items():
    prep_index = value[2]
    prep_token = value[3]
    if prep_index > 0:
      subject_ = ''
      object_ = ''
      if doc[prep_index-1].pos_ in ['NOUN','PRON'] :
        subject_ = doc[prep_index-1].text
      for child in prep_token.children:
        if child.dep_ == 'pobj':
          object_ = child.text  
      if subject_ != '' and object_ != '':
        result[key] = [subject_,object_,value[2],value[3],1]

  # 过滤调抽不出来的介词
  return_result = {}
  for key,value in result.items():
    if value[0] != '' and value[1] != '':
      return_result[key] = value
  return return_result

In [ ]:
# 合并get_verbs_sub 和 get_prep_sub 的结果，提取出三元组
def get_single_node(nodes_links,com_chunk):
  prep = ''
  if '+' in com_chunk:
    chunk_list = com_chunk.split('+')
    prep = chunk_list[0]
    chunk_list = chunk_list[1:]
  else:
    chunk_list = com_chunk.split('+')
  chunk_list_clean = []
  for chunk in chunk_list: #先把首尾的空字符串消掉
    chunk_list_clean.append(chunk.strip())

  result = ''
  print(chunk_list_clean)
  key = 0  # 设置一个阀门，防止摘出来的单词流拍
  #print(nodes_links)
  for chunk in chunk_list_clean:  # 开始补充
    for node in nodes_links:
      if chunk in node[0].split(' ') and chunk.lower() not in ['in','on','at','by','with','from','as']:  #不对介词(in,on...)补充，因为涉及词根
        result = result + node[0]+' '
        key = 1
    if chunk in ['in','on','at','by','with','from','as']:
      result = chunk + ' '
    if key == 0: #出现流拍的情况
        result = result + chunk  
  if prep != '':
    return str(prep+ ' '+result).strip()
  else:
    return result.strip()

def get_token(sentence,word):  # text -- > token 转换
  doc = nlp(sentence)
  for token in doc:
    if token.text == word:
      return token

def extend_node(sentence,word):  # text -- > texts 摘出并列词
  word_list = ''
  prep = ''
  if '+' in word:
    word_list = word.split('+')
    prep = word_list[0]
  else:
    word_list = word.split(' ')
  word_list_cleaned = []
  for ele in word_list:
    word_list_cleaned.append(ele.strip())
  
  deal_word = word_list_cleaned[-1] # 名词的词根一定是最后一个单词，所以只需要摘出这个token就可以
  result = []
  result.append(deal_word)
  #print('now,start deal with:',deal_word)
  doc = nlp(sentence)
  
  start = 0
  end = 0
  for token in doc:  # 开始找并列词
    if token.text == deal_word:
      start = 1
    elif start == 0:
      continue
    if token.dep_ == 'conj':
      if token.pos_ != 'VERB' and token.pos_ != 'ADP':
        result.append(token.text)
    if token.pos_ == 'VERB' or token.pos_ == 'ADP':
      break

  #print(result)
  if prep != '' : # 最后，如果有介词，把所有的词都加上介词
    result_prep = []
    for element in result:
      result_prep.append(prep+'+'+element)
    return result_prep
  else:
    return result


def get_triple_v2(sentence,pd_file_path):
  nodes_links = get_nodes_links(pd_file_path)
  #verb_result = get_verbs_sub(sentence)
  prep_result = get_prep_sub(sentence)
  #print(verb_result)
  #print()
  #print(prep_result)
  # 把两个字典合起来
  com_result = {}
  for key ,value in verb_result.items():
    com_result[key] = value
  for key,value in prep_result.items():
    com_result[key] =value
    
  print(com_result)
  
  # 摘取并列宾语：
  triple = []
  for key,value in com_result.items():
    sub_list = extend_node(sentence,value[0])
    obj_list = extend_node(sentence,value[1])
    for sub in sub_list:
      for obj in obj_list:
        triple.append([sub,key,obj])

  print(triple)
  # 补充节点信息
  final_triple = []
  for tri in triple:
    complete_sub = get_single_node(nodes_links,tri[0])
    complete_obj = get_single_node(nodes_links,tri[2])
    if [complete_sub,tri[1],complete_obj] not in final_triple:
      final_triple.append([complete_sub,tri[1],complete_obj])
  return final_triple

### 文本预处理：
### 1. 去掉 () 里的知识
### 2. 去掉 state-of-the-art 这种很多-的单词
### 3. 标点符号单独一格，不能和单词连在一起

In [ ]:
sentence_1 = 'with the significate progress in natural language processing and computer vision, extracting valuable information from a lot of bio medical literature has gained great popularity among researchers ,and deep learning has boosted the development of effective biomedical text mining models .'
sentence_2 = 'While BERT obtains performance comparable to that of previous models, BioBERT significantly outperforms them on the following three representative biomedical text mining tasks: biomedical named entity recognition, biomedical relation extraction and biomedical question answering .'
sentence_3 = 'We view this problem as a link prediction task in a knowledge graph, and show that a new model of the top discriminative meta paths is able to understand the meaning of some statement and accurately determine its veracity .'
sentence_4 = 'the story is that cui win the game .'
sentence_5 = 'We evaluate our approach by examining thousands of claims related to history , geography , biology, and politics using a public million node knowledge graph extracted from Wikipedia and PubMedDB .'
sentence_6 = 'Biomedical text mining is becoming increasingly important as the number of biomedical documents rapidly grows .'  # 这种没有主谓宾的结构是抽不出来的
sentence_7 = 'However , directly applying the advancements in NLP to biomedical text mining often yields unsatisfactory results due to a word distribution shift from general domain corpora to biomedical corpora .'
sentence_8 = 'With almost the same architecture across tasks , BioBERT largely outperforms BERT and previous models in a variety of biomedical text mining tasks when pre-trained on biomedical corpora .'


In [ ]:
sentence_9 = 'We expand the item response theory to study the case of cheating students for a set of exams , trying to detect cheating students by applying a greedy algorithm of inference . a greedy algorithm of inference is closely related to the Boltzmann machine learning .'


In [45]:
doc = nlp(sentence_2)
#displacy.render(doc, style='dep', jupyter=True,options = {'distance': 120})
#pf_file = './sentence_9.csv'
#print(pd.read_csv(pf_file))
#get_verbs_sub(sentence_5)
#get_triple_v2(sentence_9,pf_file)

In [46]:
#pd.read_csv(pf_file)
displacy.render(doc, style='dep', jupyter=True,options = {'distance': 120})

In [ ]:
pd.read_csv(pf_file)

,Document#,Sentence#,Word#,Word,EntityType,EntityIOB,Lemma,POS,POSTag,Start,End,Dependency
0,1,0,0,With,NaN,O,with,ADP,IN,0,3,prep
1,1,0,1,almost the same architecture,ENTITY,O,almost,ADV,RB,5,32,pobj
2,1,0,2,across,NaN,O,across,ADP,IN,34,39,prep
3,1,0,3,tasks,ENTITY,O,task,NOUN,NNS,41,45,pobj
4,1,0,4,",",NaN,O,",",PUNCT,",",46,46,punct
5,1,0,5,BioBERT,ENTITY,O,BioBERT,NOUN,NNS,48,54,nsubj
6,1,0,6,largely,NaN,O,largely,ADV,RB,56,62,advmod
7,1,0,7,outperforms,NaN,O,outperform,VERB,VBZ,64,74,ROOT
8,1,0,8,BERT,NaN,O,bert,ADJ,JJ,76,79,dobj
9,1,0,9,and,NaN,O,and,CCONJ,CC,81,83,cc


In [ ]:
doc = nlp(sentence_2)
for token in doc:
  print(token.text,token.dep_)
  '''
  if token.text == 'answering':
    print('The token:',token.text,token.dep_,token.pos_)
    print('head:')
    print(token.head.text,token.head.dep_)
    print()
    print('parent:')
    for parent in token.ancestors:
      print(parent.text,parent.dep_)
    print()
    print('childeren:')
    for child in token.children:
      print(child.text,child.dep_)
  '''

While mark
BERT nsubj
obtains advcl
performance dobj
comparable amod
to prep
that pobj
of prep
previous amod
models pobj
, punct
BioBERT nsubj
significantly advmod
outperforms ROOT
them dobj
on prep
the det
following amod
three nummod
representative amod
biomedical amod
text compound
mining compound
tasks pobj
: punct
biomedical amod
named amod
entity compound
recognition appos
, punct
biomedical amod
relation compound
extraction conj
and cc
biomedical amod
question conj
answering advcl
. punct


In [ ]:
# 打印名词块
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)
print()

# 打印依存树
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
